# Semi Supervised Learning
<br>
When you have labeled and unlabeled data you might as well use the unlabeled data to increase the predictive strength of your model. In this case you will make the following assumptions:
<br>
* The labeled samples are representative of the data (i.e. they are not outliers)
* All instances are drawn from an i.i.d. distribution
<br>

So that $p(x) = \sum_y p(x|y)p(y)$ 
<br>

There are two types of semi-supervised learning:
<br>

* **inductive** - continually learns from data, i.e. learns $\hat{y} = \hat{f}(x)$ over all feature space X
* **transductive** - predicts the labels on the unlabeled instances of the training data. So it learns $\hat{y_j} = \hat{f}(x_j \forall x_j \in D_U)$
<br>

### Self training algorithms
Use labeled data to find $\hat{f}(x^{(U)})$. Then use/consider $\hat{f}(x^{(U)})$ as "labeled" data. Some examples include:
<br>
  * **Self training algorithm** - The algorithm takes the highest confidence estimate(s) in the unlabeled data and moves it(them) to the labeled data set, thereby iteratively increasing the labeled data set. 
  * **Propagating 1-NN** - Use a distance function to iteratively merge the unlabeled data to the labeled data set.
    * If you imagine labeled and unlabeled points in 2D, this algorithm operates on the closest pair consisting of an unlabled and a labeled point. The unlabeled data point is absorbed into the labeled set and the process repeats.
    * This algorithm works well when the data is dense and well seprated for each class. 
<br><br>

### Mixture Models
Note that [the following video](https://www.youtube.com/watch?v=REypj2sy_5U&list=PLBv09BD7ez_4e9LtmK626Evn1ion6ynrt) has a great explanation of mixture models and their use. Recall that there are (at least) two types of clustering methods. 
* You have hard clustering where an element either belongs to a cluster or not. 
* You have soft clustering where an element may be associated with different clusters albeit with differing degrees of confidence.
<br>

So given data that has a higher kurtosis than the Gaussian distribution or if it appears multimodal, then chances are that the data is comprised of multiple classes. Mixture models are a probabilistically sound way of doing soft clustering with the Gaussian Mixture Model (GMM) being a common choice.
![multimodal distribution](multimodal.jpg)
<br>

Goal: We want to find $p(y|\underline{x})$<br>
Process: 
  * Model each class as a specified density with unknown parameters
  $$p(\underline{x},y|\underline{\theta}) = p(\underline{x}|y,\underline{\theta})p(y|\underline{\theta})\\
  = p(\underline{x}|y,\underline{\theta})p(y)\\
  = \text{class-conditional density * prior}$$
  <br>
  
  * for the labeled data we have $p(\underline{x}|y=1,\underline{\theta})$ and $p(\underline{x}|y=2,\underline{\theta})$
  <br>
  ![mixture model](mixture-model.jpg)
  <br>
  
  * but for the unlabeled data you know know the class for y so we have
  $$p(\underline{x}|\underline{\theta}) = \sum_{y=1}^C p(\underline{x}|y,\underline{\theta})p(y|\underline{\theta})\\
  =\sum_{y=1}^C p(\underline{x}|y,\underline{\theta})p(y)\\
  = \text{component densities * mixing parameters}
  $$
  <br>
  
Here $p(\underline{x}|\underline{\theta})$ is a mixture density. But how do we use the labeled and unlabeled data together? We can find $\underline{\theta}$ from the data using MLE. 
<br>
$$\hat{\theta_{MLE}} = \underset{\theta}{\mathrm{argmax}}\quad p(D|\theta)\\
= \underset{\theta}{\mathrm{argmax}} \space ln[p(D|\theta)]\\
= \underset{\theta}{\mathrm{argmax}} \sum_{i=1}^{l} ln[p(x_i,y_i|\theta)] + \sum_{i=l+1}^{l+u} ln[p(x_i|\theta]\\
= \underset{\theta}{\mathrm{argmax}} \sum_{i=1}^{l} ln[p(x_i|y_i\theta) + ln[p(y_i)] + \sum_{i=l+1}^{l+u} ln[\sum_{y=1}^C p(\underline{x}|y,\underline{\theta})p(y)]
$$
<br>
where:<br>

- the labeled data goes from i=1 to l, and the unlabeled data goes from i=l+1 to l+u
- $ p(y_i|\theta)=p(y_i)$ since $\theta$ doesn't give you information about y 
- remember that for the unlabeled data $p(\underline{x}|\underline{\theta})=\sum_{y=1}^C p(\underline{x}|y,\underline{\theta})p(y)$
<br>

Finally, remember that the classes as well as the parameters $\underline{\theta}$ are unknown and that you need one to estimate the other. So how do you estimtate the classes and the parameters? That is where expectation maximization (EM) comes into the picture.
<br>

### Expectation Maximization

Let's begin by listing below the comon uses for EM. Note that the second one is exactly what we are trying to figure out in the mixture model above. 
* Finding missing data
* Find MLE in difficult situations
* Estimate quantities in mixture models
<br>

Let *D* be all of the data where *D* = {$(x_i,y_i), i=1,2,...,l; x_h, h=l+1, l+2,...,l+u$} and *H* = {$y_h, h=l+1, l+2,...,l+u$} are the 'hidden' labels. Here is how the EM algorithm works:
1. initialize t=0 (iteration index) and $\underline{\theta}^0$. Theta from the labeled data is a good starting point for $\underline{\theta}^0$
2. Compute the best estimate of *H* as $p(H|D, \underline{\theta}^{(t)})$
3. Estimate parameter $\underline{\theta}^{(t+1)}$ by 
    $$\theta^{(t+1)} = \underset{\theta}{\mathrm{argmax}} \quad E_{H|D,\underline{\theta^{(t)}}}\{ ln[p(D,H|\underline{\theta}]\} $$
<br>

where steps 2 and 3 are repeated in succession until convergence is reached or a halting condition is reached. Note that EM has the following properties:<br>
- Can be shown that p(D|$\theta$) increases at each iteration
- Converges to **local optimum**
- The result depends on the starting point of $\underline{\theta}^0$. As mentioned previously, a common choice is $\hat{\underline{\theta}}_{MLE}$ using the labeled data set.
<br>

So the EM algorithm requires a good starting point and if it receives one it should iteratively get closer to discerning the classes and parameters for the unlabeled data set. The graphic below should give an intuition for EM.

![expectation maximization](https://upload.wikimedia.org/wikipedia/commons/6/69/EM_Clustering_of_Old_Faithful_data.gif)

### Resources
* Great video on EM: https://www.youtube.com/watch?v=iQoXFmbXRJA&index=2&list=PLBv09BD7ez_4e9LtmK626Evn1ion6ynrt